In [3]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import RDConfig
import os
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem import rdDepictor

In [4]:
from rdkit.Chem import PyMol
import py3Dmol
from rdkit.Chem import rdDistGeom
from rdkit.Chem import rdMolAlign

In [26]:
mol = Chem.MolFromSmiles("ClC1=CC(O[C@H]2C(C)(C)[C@H](NC(C3=CC=C(C#CC4CCN(C(C[C@H](NC([C@@H]5C[C@@H](O)CN5C([C@@H](C6=CC(C)=NO6)C(C)C)=O)=O)C7=CC=C(C8=C(C)N=CS8)C=C7)=O)CC4)C=C3)=O)C2(C)C)=CC=C1C#N")

In [27]:
mol2 = Chem.AddHs(mol)
params = AllChem.ETKDGv3()
cids = AllChem.EmbedMultipleConfs(mol2, 25, params)
res = AllChem.MMFFOptimizeMoleculeConfs(mol2)

In [28]:
print(mol2.GetNumConformers())

25


In [29]:
IPythonConsole.drawMol3D(mol2, confId=2)
IPythonConsole.drawMol3D(mol2, confId=3)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [30]:
def drawit(m, cids=[-1], p=None, removeHs=False, s=True,
           colors=('grayCarbon','grayCarbon','grayCarbon','magentaCarbon','whiteCarbon','purpleCarbon')):
        if removeHs:
            m = Chem.RemoveHs(m)
        if p is None:
            p = py3Dmol.view(width=400, height=400)
        p.removeAllModels()
        if s==False:
            for i,cid in enumerate(cids):
                IPythonConsole.addMolToView(m,p,confId=cid)
                #p.addSurface(py3Dmol.VDW,{'opacity':0.5,'model':10,'color':'gray'})
                IPythonConsole.drawOptions.highlightRadius = 0.5
            for i,cid in enumerate(cids):
                #print(cid)
                IPythonConsole.addMolToView(m,p,confId=1)
                #p.addSurface(py3Dmol.VDW,{'opacity':0.3,'model':10,'color':'gray'})
        for i,cid in enumerate(cids):
            p.setStyle({'model':i,},
                            {'stick':{'colorscheme':colors[i%len(colors)]}})
        if s:
            print(i)
        p.zoomTo()
        return p.show()

In [31]:
# Align by substructure (COCCOCC)
core = Chem.MolFromSmiles('C1(C#CC2CC[N]CC2)=CC=CC=C1')
rdMolAlign.AlignMolConformers(mol2,atomIds = mol2.GetSubstructMatch(core))
#rdMolAlign.AlignMol(mol2,mol2,prbCid=cids[1],refCid=cids[5])
#rdMolAlign.AlignMol(mol2,mol2,prbCid=cids[5],refCid=cids[9])
drawit(mol2,cids=[1,5,10,20], s=False, removeHs=True)
drawit(mol2,cids=[1], s=False, removeHs=False)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
def smiles_to_svg(smiles: str, width: int = 400, height: int = 400) -> bytes:
    """
    makes an SVG image of a molecule
    """
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise RuntimeError("Invalid SMILES")

    Chem.rdCoordGen.AddCoords(mol)
    drawer = Chem.Draw.rdMolDraw2D.MolDraw2DSVG(width, height)
    # set drawing options on drawer.getOptions()
    drawer.DrawMolecule(mol)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText()

In [17]:
smiles_to_svg("COC1=CC(C(C2=C3C=NC=C2)=CN(C3=O)C)=CC(OC)=C1CN4CCN(CC4)CCOCCOCCOC5=CC(C6=C(N=CS6)C)=CC=C5CNC([C@@H]7C[C@H](CN7C([C@@H](NC(C8(CC8)F)=O)C(C)(C)C)=O)O)=O")

b"<?xml version='1.0' encoding='iso-8859-1'?>\n<svg version='1.1' baseProfile='full'\n              xmlns='http://www.w3.org/2000/svg'\n                      xmlns:rdkit='http://www.rdkit.org/xml'\n                      xmlns:xlink='http://www.w3.org/1999/xlink'\n                  xml:space='preserve'\nwidth='400px' height='400px' viewBox='0 0 400 400'>\n<!-- END OF HEADER -->\n<rect style='opacity:1.0;fill:#FFFFFF;stroke:none' width='400.0' height='400.0' x='0.0' y='0.0'> </rect>\n<path class='bond-0 atom-0 atom-1' d='M 129.5,135.9 L 128.3,141.9' style='fill:none;fill-rule:evenodd;stroke:#000000;stroke-width:2.0px;stroke-linecap:butt;stroke-linejoin:miter;stroke-opacity:1' />\n<path class='bond-0 atom-0 atom-1' d='M 128.3,141.9 L 127.0,147.9' style='fill:none;fill-rule:evenodd;stroke:#FF0000;stroke-width:2.0px;stroke-linecap:butt;stroke-linejoin:miter;stroke-opacity:1' />\n<path class='bond-1 atom-1 atom-2' d='M 122.3,153.3 L 116.3,155.2' style='fill:none;fill-rule:evenodd;stroke:#FF0

In [26]:
def smiles_to_svg(
        smiles: str,
        save_path: "/home/user2/kinaseHTGS/",
        width: int = 600,
        height: int = 600
) -> None:
    """Converts a SMILES string to an SVG image of the molecule.

    :param smiles: A SMILES string.
    :param save_path: Path to an SVG file where the molecule image will be saved.
    :param width: Width of the SVG image.
    :param height: Height of the SVG image.
    """
    # Convert SMILES to Mol
    mol = Chem.MolFromSmiles(smiles)

    # Convert Mol to SVG
    d = Chem.Draw.rdMolDraw2D.MolDraw2DSVG(width, height)
    d.DrawMolecule(mol)
    d.FinishDrawing()
    svg = d.GetDrawingText().encode()

    # Save SVG
    #save_path.parent.mkdir(parents=True, exist_ok=True)
    save_path.write_text(svg)